# Getting Started with DSPy

This notebook introduces the fundamental concepts of DSPy:
- Setting up language models
- Creating signatures
- Using basic modules
- Making predictions

DSPy is a framework for algorithmically optimizing LM prompts and weights, especially when LMs are used one or more times within a pipeline.

## Setup

First, let's import the necessary libraries and set up our environment.

In [1]:
import os
import sys
sys.path.append('../../')

import dspy
from utils import setup_default_lm, print_step, print_result, print_error
from dotenv import load_dotenv

# Load environment variables
load_dotenv('../../.env')

False

## Language Model Configuration

DSPy supports various language models. Let's configure one for our examples.

In [2]:
print_step("Setting up Language Model", "Configuring DSPy with OpenAI gpt-4o")

try:
    # Set up the language model
    lm = setup_default_lm(provider="openai", model="gpt-4o", max_tokens=500)
    
    # Configure DSPy to use this model
    dspy.configure(lm=lm)
    
    print_result("Language model configured successfully!", "Status")
    
except Exception as e:
    print_error(f"Failed to configure language model: {e}")
    print("Make sure you have set your OPENAI_API_KEY in the .env file")

=== Setting up Language Model ===
Configuring DSPy with OpenAI gpt-4o

Result:
Successfully configured openai language model

Status:
Language model configured successfully!



## DSPy Signatures

Signatures define the input/output behavior of your language model calls. They're like type hints for LM operations.

In [3]:
print_step("Creating DSPy Signatures", "Defining input/output specifications")

# Simple question answering signature
class QuestionAnswering(dspy.Signature):
    """Answer the given question with a concise and accurate response."""
    question = dspy.InputField(desc="The question to be answered")
    answer = dspy.OutputField(desc="A concise answer to the question")

# Text classification signature
class SentimentClassification(dspy.Signature):
    """Classify the sentiment of the given text as positive, negative, or neutral."""
    text = dspy.InputField(desc="The text to classify")
    sentiment = dspy.OutputField(desc="The sentiment: positive, negative, or neutral")

print_result("Signatures created successfully!")

=== Creating DSPy Signatures ===
Defining input/output specifications

Result:
Signatures created successfully!



## Basic Prediction Module

The `Predict` module is the simplest way to use a signature with a language model.

In [4]:
print_step("Using Predict Module", "Making basic predictions with our signatures")

# Create prediction modules
qa_predictor = dspy.Predict(QuestionAnswering)
sentiment_predictor = dspy.Predict(SentimentClassification)

# Test question answering
question = "What is the capital of France?"
qa_result = qa_predictor(question=question)

print_result(f"Question: {question}\nAnswer: {qa_result.answer}", "Question Answering")

# Test sentiment classification
text = "I absolutely love this new product! It's fantastic!"
sentiment_result = sentiment_predictor(text=text)

print_result(f"Text: {text}\nSentiment: {sentiment_result.sentiment}", "Sentiment Classification")

=== Using Predict Module ===
Making basic predictions with our signatures

Question Answering:
Question: What is the capital of France?
Answer: Paris

Sentiment Classification:
Text: I absolutely love this new product! It's fantastic!
Sentiment: positive



## Chain of Thought Reasoning

The `ChainOfThought` module adds reasoning steps before providing the final answer.

In [5]:
print_step("Using ChainOfThought Module", "Adding reasoning steps to predictions")

# Create a math reasoning signature
class MathReasoning(dspy.Signature):
    """Solve the mathematical problem step by step."""
    problem = dspy.InputField(desc="The mathematical problem to solve")
    reasoning = dspy.OutputField(desc="Step-by-step reasoning")
    answer = dspy.OutputField(desc="The final numerical answer")

# Use ChainOfThought for better reasoning
math_cot = dspy.ChainOfThought(MathReasoning)

# Test with a math problem
problem = "If a rectangle has a length of 8 meters and a width of 5 meters, what is its area?"
math_result = math_cot(problem=problem)

print_result(f"Problem: {problem}\nReasoning: {math_result.reasoning}\nAnswer: {math_result.answer}", "Math Reasoning")

=== Using ChainOfThought Module ===
Adding reasoning steps to predictions

Math Reasoning:
Problem: If a rectangle has a length of 8 meters and a width of 5 meters, what is its area?
Reasoning: To find the area of a rectangle, we use the formula:

\[ \text{Area} = \text{Length} \times \text{Width} \]

In this problem, the length of the rectangle is given as 8 meters, and the width is given as 5 meters. Substituting these values into the formula, we have:

\[ \text{Area} = 8 \, \text{meters} \times 5 \, \text{meters} \]

Calculating this gives:

\[ \text{Area} = 40 \, \text{square meters} \]

Therefore, the area of the rectangle is 40 square meters.
Answer: 40



## Custom DSPy Module

You can create custom modules by subclassing `dspy.Module`.

In [6]:
print_step("Creating Custom Module", "Building a comprehensive question answering system")

class SmartQA(dspy.Module):
    def __init__(self):
        super().__init__()
        
        # Define signature for classification
        class QuestionType(dspy.Signature):
            """Classify the type of question being asked."""
            question = dspy.InputField(desc="The question to classify")
            question_type = dspy.OutputField(desc="Type: factual, mathematical, creative, or analytical")
        
        # Define signature for answering
        class AnswerQuestion(dspy.Signature):
            """Answer the question based on its type."""
            question = dspy.InputField(desc="The question to answer")
            question_type = dspy.InputField(desc="The type of question")
            answer = dspy.OutputField(desc="A comprehensive answer")
        
        # Initialize modules
        self.classify_question = dspy.Predict(QuestionType)
        self.answer_question = dspy.ChainOfThought(AnswerQuestion)
    
    def forward(self, question):
        # First, classify the question type
        classification = self.classify_question(question=question)
        
        # Then answer based on the type
        answer = self.answer_question(
            question=question,
            question_type=classification.question_type
        )
        
        return dspy.Prediction(
            question_type=classification.question_type,
            reasoning=answer.reasoning,
            answer=answer.answer
        )

# Create and test the custom module
smart_qa = SmartQA()

test_questions = [
    "What is the speed of light?",
    "If I have 10 apples and eat 3, how many do I have left?",
    "Write a creative story about a robot learning to paint.",
]

for question in test_questions:
    result = smart_qa(question=question)
    print_result(
        f"Question: {question}\n"
        f"Type: {result.question_type}\n"
        f"Reasoning: {result.reasoning}\n"
        f"Answer: {result.answer}",
        f"Smart QA Result"
    )
    print("-" * 80)

=== Creating Custom Module ===
Building a comprehensive question answering system

Smart QA Result:
Question: What is the speed of light?
Type: factual
Reasoning: The speed of light is a fundamental constant in physics, often denoted by the symbol "c". It is the speed at which light travels in a vacuum and is a crucial component in many areas of physics, including the theory of relativity. The value of the speed of light is well-established and widely accepted in the scientific community.
Answer: The speed of light in a vacuum is approximately 299,792,458 meters per second (m/s).

--------------------------------------------------------------------------------
Smart QA Result:
Question: If I have 10 apples and eat 3, how many do I have left?
Type: mathematical
Reasoning: To determine how many apples are left after eating some, we need to subtract the number of apples eaten from the total number of apples initially possessed. You start with 10 apples and eat 3 of them. Therefore, the ca

2025/11/27 09:41:44 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=500. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.


Smart QA Result:
Question: Write a creative story about a robot learning to paint.
Type: creative
Reasoning: To craft a creative story about a robot learning to paint, I will focus on developing a narrative that explores themes of discovery, creativity, and the intersection of technology and art. The story will involve a robot protagonist who embarks on a journey of self-expression, encountering challenges and moments of inspiration along the way. This will allow for a rich exploration of the robot's evolving understanding of art and its impact on its identity.
Answer: In a bustling city where technology thrived and skyscrapers kissed the clouds, there existed a small, unassuming workshop nestled between towering buildings. Inside, an inventor named Dr. Elara spent her days creating machines that could think, learn, and adapt. Her latest creation was a robot named Arti, designed with the ability to learn and mimic human behaviors.

Arti was a sleek, silver machine with a curious mind. 

## Working with Examples

DSPy uses `Example` objects to represent training and evaluation data.

In [7]:
print_step("Working with Examples", "Creating and using DSPy Example objects")

# Create examples
examples = [
    dspy.Example(question="What is 2+2?", answer="4"),
    dspy.Example(question="Who wrote Romeo and Juliet?", answer="William Shakespeare"),
    dspy.Example(question="What is the largest planet?", answer="Jupiter"),
]

print_result(f"Created {len(examples)} examples")

# Test our QA predictor on these examples
print("Testing predictor on examples:")
for i, example in enumerate(examples, 1):
    prediction = qa_predictor(question=example.question)
    print(f"\nExample {i}:")
    print(f"Question: {example.question}")
    print(f"Expected: {example.answer}")
    print(f"Predicted: {prediction.answer}")
    print(f"Match: {prediction.answer.lower().strip() == example.answer.lower().strip()}")

=== Working with Examples ===
Creating and using DSPy Example objects

Result:
Created 3 examples

Testing predictor on examples:

Example 1:
Question: What is 2+2?
Expected: 4
Predicted: 4
Match: True

Example 2:
Question: Who wrote Romeo and Juliet?
Expected: William Shakespeare
Predicted: William Shakespeare
Match: True

Example 3:
Question: What is the largest planet?
Expected: Jupiter
Predicted: Jupiter is the largest planet in our solar system.
Match: False


## Inspecting LM Calls

DSPy allows you to inspect the actual prompts and responses sent to the language model.

In [8]:
print_step("Inspecting LM History", "Looking at prompts and responses")

# Make a prediction to generate history
result = qa_predictor(question="What is machine learning?")

# Inspect the history
if hasattr(lm, 'history') and lm.history:
    latest_call = lm.history[-1]
    print_result(
        f"Prompt: {latest_call.get('prompt', 'N/A')}\n\n"
        f"Response: {latest_call.get('response', 'N/A')}",
        "Latest LM Call"
    )
else:
    print_result("History not available for this LM configuration", "Note")

print_result(f"Answer: {result.answer}", "Final Result")

=== Inspecting LM History ===
Looking at prompts and responses

Latest LM Call:
Prompt: None

Response: ModelResponse(id='chatcmpl-CgLJouo41CPVf64esZTIy3HzLW7tj', created=1764207760, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_e819e3438b', choices=[Choices(finish_reason='stop', index=0, message=Message(content='[[ ## answer ## ]]\nMachine learning is a subset of artificial intelligence that involves the use of algorithms and statistical models to enable computers to improve their performance on a task through experience and data, without being explicitly programmed for that task.\n\n[[ ## completed ## ]]', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=50, prompt_tokens=153, total_tokens=203, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_pred

## Summary

In this notebook, we covered:

1. **Language Model Setup**: How to configure DSPy with different LM providers
2. **Signatures**: Defining input/output specifications for LM operations
3. **Basic Modules**: Using `Predict` for simple predictions
4. **Chain of Thought**: Adding reasoning steps with `ChainOfThought`
5. **Custom Modules**: Creating complex workflows by subclassing `dspy.Module`
6. **Examples**: Working with training/evaluation data
7. **Inspection**: Understanding what's happening under the hood

These are the fundamental building blocks for creating more sophisticated DSPy applications. In the next notebooks, we'll explore optimization, retrieval-augmented generation, and advanced techniques.